In [18]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [46]:
doc_1 = " MetricsDB has 3 main components. Cluster Manager is responsible for assigning partitions to backend servers. HDFS is used to store mappings from partitions to servers. Backend Servers are responsible for processing metrics for a small number of partitions. Each Backend Server keeps the latest two hours of data for all metrics in memory and they checkpoint in-memory data every two hours to durable storage, Blobstore.  Coordinators are responsible for routing requests to all replica sets and validates desired quorum constraints are met.  MetricsDB is multi-zone compliant. MetricsDB has a replication factor of three. Apache Kafka producer has an option to batch requests, which helped us reduce the number of requests to the queue and storage."
#Resp 03 and 10:
doc_2 = "The diagram describes a system architecture in a simple overview. The entity cordinator access the Kafka platform as the the replicas sets do too. Each replica set is composed by two elements: the cluster manager that access the HDFS dabase and the Backend Servers that access the Blobstore database. "
doc_3 = "coordinator use kafka message service that has to distribute the transactions to hadoop and blob storage service"

doc_4 = "Whenever an event occurs in the Spotify client, it is sent to one of the Spotify gateways which logs it via syslog. There it is assigned a timestamp that is used throughout the event delivery system.  all data needs to be delivered to a centrally located Hadoop cluster. All data that gets delivered via our event delivery service is written on HDFS. The File Tailer tails log files looking for new events, and forwards them to the Event Delivery Service. As soon as it gets a confirmation that the event has been received it’s responsibility ends. The Event Delivery Service accepts events from the Tailer, transform them to their final structured format and forwards them to the Kafka Brokers. It is built as a RESTful microservice using the Apollo framework and deployed using the Helios orchestration platform, a common design pattern at Spotify. Embedding a simple Kafka producer in the Event Delivery Service also proved to be easy. The Mirror Maker project introduced mirroring between data centers, and the Camus project can be used for exporting Avro structured events to hourly buckets."
doc_5 = "DBLog is a Generic Change-Data-Capture Framework that allows capturing committed changes from a database in real-time and propagating those changes to downstream consumers. Transaction logs from databases are the source of CDC events. DBLog is a Java-based framework, able to capture changes in real-time and to take dumps. We use Zookeeper to store state related to log and dump processing, and for leader election. In order to use DBLog a database needs to provide a change log from a linear history of committed changes and these conditions are fulfilled by systems like MySQL, PostgreSQL, MariaDB, etc. Dump processing was integrated by using SQL and JDBC, only requiring to implement the chunk selection and watermark update. The same code is used for MySQL and PostgreSQL and can be used for other similar databases as well. DBLog uses an active-passive architecture. One instance is active and the others are passive standbys. We leverage Zookeeper for leader election to determine the active instance."

corpus = np.array([doc_1, doc_2, doc_3, doc_4, doc_5])

### 1) Aplicar lowercase e remover caracteres

In [47]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [48]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['metricsdb has main components cluster manager is responsible for assigning partitions to backend servers hdfs is used to store mappings from partitions to servers backend servers are responsible for processing metrics for small number of partitions each backend server keeps the latest two hours of data for all metrics in memory and they checkpoint inmemory data every two hours to durable storage blobstore coordinators are responsible for routing requests to all replica sets and validates desired quorum constraints are met metricsdb is multizone compliant metricsdb has replication factor of three apache kafka producer has an option to batch requests which helped us reduce the number of requests to the queue and storage'
 'the diagram describes system architecture in simple overview the entity cordinator access the kafka platform as the the replicas sets do too each replica set is composed by two elements the cluster manager that access the hdfs dabase and the backend servers that acce

### 2) Remover stopwords

In [49]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [50]:
corpus_filtered = clear_stopwords(corpus_clear, "english")
print(corpus_filtered)

['metricsdb main components cluster manager responsible assigning partitions backend servers hdfs used store mappings partitions servers backend servers responsible processing metrics small number partitions backend server keeps latest two hours data metrics memory checkpoint inmemory data every two hours durable storage blobstore coordinators responsible routing requests replica sets validates desired quorum constraints met metricsdb multizone compliant metricsdb replication factor three apache kafka producer option batch requests helped us reduce number requests queue storage'
 'diagram describes system architecture simple overview entity cordinator access kafka platform replicas sets replica set composed two elements cluster manager access hdfs dabase backend servers access blobstore database'
 'coordinator use kafka message service distribute transactions hadoop blob storage service'
 'whenever event occurs spotify client sent one spotify gateways logs via syslog assigned timestamp

## Usando a biblioteca sklearn

In [51]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [52]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



/home/jalves/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able,accepts,access,active,activepassive,allows,also,apache,apollo,architecture,...,used,uses,using,validates,via,watermark,well,whenever,written,zookeeper
text_1,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
text_2,0,0,3,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
text_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
text_4,0,1,0,0,0,0,1,0,1,0,...,2,0,2,0,2,0,0,1,1,0
text_5,1,0,0,2,1,1,0,0,0,1,...,2,1,1,0,0,1,1,0,0,2


In [53]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['Diag_1','resp_1','resp_2', 'Diag_2', 'Diag_3'])

,Diag_1,resp_1,resp_2,Diag_2,Diag_3
text_1,1.000000,0.233862,0.075641,0.079498,0.036628
text_2,0.233862,1.000000,0.047565,0.080754,0.041459
text_3,0.075641,0.047565,1.000000,0.205222,0.044699
text_4,0.079498,0.080754,0.205222,1.000000,0.086729
text_5,0.036628,0.041459,0.044699,0.086729,1.000000
